<a href="https://colab.research.google.com/github/nhareesha/MLAI/blob/main/LLM/finetuned/Zeroshot_referencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faker

In [ ]:
pip install xmltodict


In [ ]:
import csv
from faker import Faker
import xmltodict
import random
import string
import datetime

faker = Faker()

def generate_ddl_insert(table_name, num_columns=5):
    columns = []
    for i in range(num_columns):
        column_name = f"column_{i+1}"
        data_type = faker.random_element(["TEXT", "TEXT", "INT", "VARCHAR(100)", "DATE", "DECIMAL(10, 2)", "TEXT", "BOOLEAN", "TIMESTAMP", "FLOAT", "DOUBLE", "CHAR(50)", "TEXT",  "TEXT", "TIMESTAMP"])
        # Set max length for TEXT columns
        if data_type == "TEXT":
            data_type = f"{data_type} (20)"
        columns.append(f"{column_name} {data_type}")

    ddl = f"CREATE TABLE {table_name} ({', '.join(columns)});"

    start_date = datetime.datetime(2010, 1, 1)
    end_date = datetime.datetime(2024, 7, 31)

    num_rows = faker.random_int(min=1, max=5) # number of insert statements
    insert_values = []
    for i in range(num_rows):
        row_values = []
        for column in columns:
            if "INT" in column:
                value = faker.random_int()
            elif "VARCHAR" in column:
                value = faker.word()
            elif "DATE" in column:
                value = faker.date()
            elif "TIMESTAMP" in column:
                value = faker.date()
            elif "DECIMAL" in column:
                value = faker.pyfloat(max_value=10000.0, min_value=-10000.0)
            elif "TEXT" in column:
                # Dynamic generation based on column name
                if "name" in column.lower():
                    value = faker.name()
                elif "firstname" in column.lower() or "fname" in column.lower():
                    value = faker.first_name()
                elif "lastname" in column.lower() or "lname" in column.lower():
                    value = faker.first_name()
                elif "address" in column.lower():
                    value = faker.address()
                elif "city" in column.lower():
                    value = faker.city()
                elif "json" in column.lower():
                    value = faker.json()
                elif "XML" in column.lower():
                    xml_data = faker.json()
                    value = xmltodict.unparse(xml_data)
                else:
                    value = generate_random_word(random.randint(4, 10))

            elif "BOOLEAN" in column:
                value = faker.boolean()
            elif "TIMESTAMP" in column:
                value = faker.date_time()
            elif "FLOAT" in column:
                value = faker.pyfloat()
            elif "DOUBLE" in column:
                value = faker.pyfloat()
            elif "CHAR" in column:
                value = faker.word()[:50]  # Ensure length doesn't exceed CHAR limit
            else:
                value = generate_random_word(random.randint(4, 10))  # Default for unknown data types
            row_values.append(value)
        insert_values.append(f"({', '.join(str(value) for value in row_values)})")

    insert = f"INSERT INTO {table_name} VALUES {', '.join(insert_values)};"

    return ddl, insert

def generate_random_word(length=10):
  """Generates a random word with no meaning.

  Args:
    length: The desired length of the word.

  Returns:
    A random word with no meaning.
  """

  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for _ in range(length))


In [ ]:
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Write to a CSV file on your Google Drive
csv_path = '/content/drive/MyDrive/diverse_test_data_01.csv'  # Replace with your desired path


num_tables = 400000
with open(csv_path, 'a', newline='') as csvfile:
    fieldnames = ['ddl', 'insert']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if csvfile.tell() == 0:
      writer.writeheader()

    for i in range(num_tables):
        table_name = f"table_{i+1}"
        column_number = random.randint(4, 15)
        ddl, insert = generate_ddl_insert(table_name, column_number)
        writer.writerow({'ddl': ddl, 'insert': insert})

print("Fintech dataset with diverse schemas created successfully!")

Fintech dataset with diverse schemas created successfully!


In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
csv_path = '/content/drive/MyDrive/diverse_test_data_01.csv'

In [ ]:

# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")


def generate_ddl_insert(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Create a DDL statement for a table named \"customers\" with columns for customer ID, name, email, and phone number. Include an INSERT statement with sample data for 3 customers."
generated_ddl_insert = generate_ddl_insert(prompt)
print(generated_ddl_insert)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

a table named "customers" with columns for customer ID, name, email, and phone number. Include an INSERT statement with sample data for 3 customers.
